In [26]:
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
import numpy as np

from glob import glob
import random
import os

Data preparation

In [2]:
import pandas as pd
tw=pd.read_excel('rnn.xlsx')
tw=tw[['text','sentiment']]


In [3]:
# tokenize the dataset corpus, delete uncommon words such as names, etc.
    tokenizer = Tokenizer(num_words=2000)
    tokenizer.fit_on_texts(tw['text'])
    X = tokenizer.texts_to_sequences(tw['text'])
    X = np.array(X)
    # pad sequences with 0's
    X = pad_sequences(X)


In [4]:
Y = pd.get_dummies(tw['sentiment']).values
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
data = {}
data["X_train"] = X_train
data["X_test"]= X_test
data["y_train"] = y_train
data["y_test"] = y_test
data["tokenizer"] = tokenizer
data["int2label"] =  {0: "negative", 1: "positive"}
data["label2int"] = {"negative": 0, "positive": 1}

pre-trained GloVe

In [ ]:
def get_embedding_vectors(word_index, embedding_size=100):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_size))
    with open(f"data/glove.6B.{embedding_size}d.txt", encoding="utf8") as f:
        for line in tqdm(f, "Reading GloVe"):
            values = line.split()
            # get the word as the first word in the line
            word = values[0]
            if word in word_index:
                idx = word_index[word]
                # get the vectors as the remaining values in the line
                embedding_matrix[idx] = np.array(values[1:], dtype="float32")
    return embedding_matrix

function that creates the model 

training data

In [ ]:
# max number of words in each sentence
SEQUENCE_LENGTH = 300
# N-Dimensional GloVe embedding vectors
EMBEDDING_SIZE = 300
# number of words to use, discarding the rest
N_WORDS = 10000
# out of vocabulary token
OOV_TOKEN = None
# 30% testing set, 70% training set
TEST_SIZE = 0.3
# number of CELL layers
N_LAYERS = 1
# the RNN cell to use, LSTM in this case
RNN_CELL = LSTM
# whether it's a bidirectional RNN
IS_BIDIRECTIONAL = False
# number of units (RNN_CELL ,nodes) in each layer
UNITS = 128
# dropout rate
DROPOUT = 0.4
### Training parameters
LOSS = "categorical_crossentropy"
OPTIMIZER = "adam"
BATCH_SIZE = 64
EPOCHS = 6

max_fatures= 2000
embed_dim = 128
lstm_out=196
dropout=0.2
recurrent_dropout=0.2
loss="categorical_crossentropy"
optimizer="adam"

#word_index
embedding_size=100

training our model:

In [5]:
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

from keras.preprocessing.text import text_to_word_sequence
import re

Using TensorFlow backend.


In [6]:

# create these folders if they does not exist
if not os.path.isdir("results"):
    os.mkdir("results")



In [10]:
max_fatures= 2000
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 58, 128)           256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 58, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
def create_model(word_index,embedding_size=100,  max_fatures= 2000,
                 embed_dim = 128,lstm_out=196, dropout=0.2,
                 recurrent_dropout=0.2,loss="categorical_crossentropy", optimizer="adam"):
        
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss, optimizer, metrics = ['accuracy'])
    
    print(model.summary())

In [ ]:
# construct the model
model = create_model(data["tokenizer"].word_index, embedding_size=100, max_fatures= 2000, 
                     embed_dim = 128,lstm_out=196, dropout=0.2,recurrent_dropout=0.2,
                     loss="categorical_crossentropy", optimizer="adam")

    
model.summary()


In [13]:
# start training
batch_size = 32
history=model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 3)



Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5


In [28]:
# save the resulting model into 'results' folder
model.save(os.path.join("results", "modelo_rnn") + ".h5")


testing

In [18]:
SEQUENCE_LENGTH = 58

def get_predictions(text):
    sequence = data["tokenizer"].texts_to_sequences([text])
    # pad the sequences
    sequence = pad_sequences(sequence, maxlen=SEQUENCE_LENGTH)
    # get the prediction
    prediction = model.predict(sequence)[0]
    return prediction, data["int2label"][np.argmax(prediction)]

In [27]:

text = "this is the best company"
output_vector, prediction = get_predictions(text)
print("Output vector:", output_vector)
print("Prediction:", prediction)

Output vector: [0.01299936 0.9870006 ]
Prediction: positive
